In [ ]:
import pandas as pd
import numpy as np
import unittest
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

class panda_reader():
    filename = None
    worksheetname = None
    df = None

    # Getters and setters for filename, worksheetname, and df
    def getFilename(self):
        if self.filename is None:
            logger.info('filename is not yet set!!')
            return None
        else:
            logger.debug('About to return current filename, which is: ' + self.filename)
        
        return self.filename
        
    def setFilename(self, fn):
        logger.debug('About to set filename to: ' + fn)
        self.filename = fn
        
    def getWorksheetName(self):
        return self.worksheetname
    
    def setWorksheetName(self, wks):
        self.worksheetname = wks
        
    def getDf(self):
        return self.df

    # Return a tiny test dataframe
    def test_df(self):
        data = np.array([['','Col1','Col2'],
                         ['Row1',1,2],
                         ['Row2',3,4]])
                
        return (pd.DataFrame(data=data[1:,1:], index=data[1:,0], columns=data[0,1:]))
    
    # Write the given df to the excelFileName and excelWorksheet (unless they have already been 
    # provided and are then optional.)
    def write_df_to_excel(self, df=None, excelFileName=None, excelWorksheet=None):
        # Use df (passed param) if it's set, self.df otherwise.
        if df is None:
            mydf = self.df
        else:
            mydf = df
        fn  = excelFileName or self.filename # If excelFileName is set, then use it. If None, then use self.filename.
        wks = excelWorksheet or self.worksheetname # Use the passed excelWorksheet, if available. 
        writer = pd.ExcelWriter(fn)
        mydf.to_excel(writer, wks)
        writer.save()
        
    # Read the given excelFileName and excelWorksheet (unless they have already been 
    # provided and are then optional.) return the df in the given worksheet. 
    def read_df_from_excel(self, excelFileName=None, excelWorksheet=None):
        # TODO: read this from a spreadsheet and stick it in self.df
        
        return self.df
    
class Test_panda_reader(unittest.TestCase):
    tempExcel = 'c:\\Users\\Owner\\temp\\test.xlsx'
    def testFilename(self):
        logger.info('starting testFilename.')
        testPr = panda_reader()
        testFilename = 'Fred.xls'
        testPr.setFilename(testFilename)
        self.assertEqual(testPr.getFilename(), testFilename)
        logger.info('exiting testFilename')
    
    def testWorksheetName(self):
        logger.info('starting testWorksheetName.')
        testPr = panda_reader()
        testWorksheet = 'First sheet'
        testPr.setWorksheetName(testWorksheet)
        self.assertEqual(testPr.getWorksheetName(), testWorksheet)
        logger.info('exiting testWorksheetName')
        
    def test_test_df(self):
        logger.info('starting test_test_df.')
        testPr = panda_reader()
        df = testPr.test_df()
        logger.debug('DF contains:' + df.head())
        logger.info('exiting test_test_df')
        
    def test_write_df(self):
        logger.info('Starting test_write_df.')
        testPr = panda_reader()
        tinydf = testPr.test_df()
        testPr.setFilename(self.tempExcel)
        testPr.setWorksheetName('testWks')
        testPr.write_df_to_excel(tinydf)
        logger.info('Ending test_write_df.')
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

In [8]:
import logging
import unittest
from os.path import exists, isfile, isdir, split, join
import pandas as pd
import shutil

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

"""
Class FileUtil
"""
class FileUtil:
    def read_text_file(self, filename):
        """
        Read the given file and return its lines as a list, one per line.
        """
        try:
            f = open(filename, 'r')
            lines = f.read().splitlines()
            f.close()
            f = None
            return lines
        except UnicodeDecodeError as e:
            logger.error('Unicode Decode error.')
            raise e
        except IOError as e:
            logger.error('I/O error.')
            raise e
        
    def write_text_file(self, filename, lines):
        """
        (Over)write the given filename with the given lines.
        """
        logger.debug('Starting write_text_file. Writing to %s.' % (filename))
        with open(filename, "w") as text_file:
            for line in lines:
                text_file.write("{0}\n".format(line))
                
    def append_text_file(self, filename, lines):
        """
        Append to the given filename with the given lines.
        """
        logger.debug('Starting append_text_file. Writing to %s.' % (filename))
        with open(filename, "a") as text_file:
            for line in lines:
                text_file.write("{0}\n".format(line))
                

    def load_logs_and_subdir_names(self, rootPath, requiredPrefix=None, requiredSuffix=None):
        """
        given the rootPath, traverse the subdirectories and return a list of the given prefix (if any) and suffix (if any).
        """
        logger.debug('Starting load_logs_and_subdir_names with rootPath = %s.' % (rootPath))
        ans = []
        
        for dirname, dirnames, filenames in walk(rootPath):
            # Print path to all filenames.
            cur_dir = [f.qualified_path(dirname, filename) for filename in filenames]
            ans.extend(cur_dir)
            
        if requiredPrefix:
            ans = [f for f in ans if f.startswith(requiredPrefix)]
        
        if requiredSuffix:
            ans = [f for f in ans if f.startswith(requiredSuffix)]
            
        return sorted(ans)
    
    def qualified_path(self, dirpath, filename):
        """
        From the given dir and filename, return a qualified path.
        """
        logger.debug('Starting qualified path with a dirpath of %s and a filename of %s.' % (dirpath, filename))
        return join(dirpath, filename)
    
    def split_qualified_path(self, qualified_path):
        """
        From the given qualified path, return dir and filename.
        """
        path, filename = split(qualified_path)
        logger.debug('split_qualified_path splitting %s into %s and %s.' % (qualified_path, path, filename))
        return path, filename
    
    def write_dataframe_to_csv(self, df, dirpath, filename):
        """
        Write the df to the given CSV file.
        """
        qualified_path = self.qualified_path(dirpath, filename)
        df.to_csv(qualified_path, sep=',', encoding='utf-8')
        logger.debug('write_dataframe_to_csv wrote to %s.' % qualified_path)
        
    
class Test_FileUtil(unittest.TestCase):
    path = r'C:\temp'
    fn = 'test.csv'
    txtfile = 'test.txt'
    original = ['first', 'second', 'final line']
    appendme = ['newly', 'appended']
    
    def create_csv(self):
        d = {'col1': [1,2], 'col2': [3,4]}
        df = pd.DataFrame(data=d)
        fu = FileUtil()
        fu.write_dataframe_to_csv(df, self.path, self.fn)
        logger.debug('create_csv to path %s and file %s.' % (self.path, self.fn))
        
    def test_qualified_path(self):
        logger.debug('Starting test_qualified_path.')
        fu = FileUtil()
        shouldBe = self.path + '\\' + self.fn
        self.assertEqual(fu.qualified_path(self.path, self.fn), shouldBe)
        
    def test_write_text_file(self):
        logger.debug('Starting write_text_file.')
        fu = FileUtil()
        
        fname = fu.qualified_path(self.path, self.txtfile)
        fu.write_text_file(fname, self.original)
        readin = fu.read_text_file(fname)
        self.assertEqual(self.original, readin)
        
    def test_append_text_file(self):
        logger.debug('Starting append_text_file.')
        fu = FileUtil()
        myoriginal = self.original
        
        fname = fu.qualified_path(self.path, self.txtfile)
        fu.write_text_file(fname, myoriginal)
        fu.append_text_file(fname, self.appendme)
        readin = fu.read_text_file(fname)
        myoriginal.extend(self.appendme)
        self.assertEqual(myoriginal, readin)
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    logger.info('done.')

DEBUG:__main__:Starting append_text_file.
DEBUG:__main__:Starting qualified path with a dirpath of C:\temp and a filename of test.txt.
DEBUG:__main__:Starting write_text_file. Writing to C:\temp\test.txt.
DEBUG:__main__:Starting append_text_file. Writing to C:\temp\test.txt.
.DEBUG:__main__:Starting test_qualified_path.
DEBUG:__main__:Starting qualified path with a dirpath of C:\temp and a filename of test.csv.
.DEBUG:__main__:Starting write_text_file.
DEBUG:__main__:Starting qualified path with a dirpath of C:\temp and a filename of test.txt.
DEBUG:__main__:Starting write_text_file. Writing to C:\temp\test.txt.
.
----------------------------------------------------------------------
Ran 3 tests in 0.065s

OK
INFO:__main__:done.


In [7]:
self.original


NameError: name 'self' is not defined